En este archivo crearemos un dataframe al estilo doomies de los generos que poseemos.

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [41]:
data = pd.read_csv('data/peliculas_ETL.csv', low_memory=False, index_col='id')
df = data.copy()
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42196 entries, 862 to 461257
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  42196 non-null  object 
 1   budget                 42196 non-null  float64
 2   genres                 42196 non-null  object 
 3   original_language      42196 non-null  object 
 4   overview               42196 non-null  object 
 5   popularity             42196 non-null  float64
 6   production_companies   42196 non-null  object 
 7   production_countries   42196 non-null  object 
 8   release_date           42196 non-null  object 
 9   revenue                42196 non-null  float64
 10  runtime                42196 non-null  float64
 11  spoken_languages       42196 non-null  object 
 12  status                 42196 non-null  object 
 13  tagline                42196 non-null  object 
 14  title                  42196 non-null  object 
 15  vote

In [42]:
df.reset_index()

,id,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,director,actors_names,release_year,return
0,862,Toy Story Collection,30000000.0,"['Animation', 'Comedy', 'Family']",en,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,['Pixar Animation Studios'],['United States of America'],1995-10-30,373554033.0,81.0,['English'],Released,N/D,Toy Story,7.7,5415.0,John Lasseter,"['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim Varney', 'Wallace Shawn', 'John Ratzenberger', 'Annie Potts', 'John Morris', 'Erik von Detten', 'Laurie Metcalf', 'R. Lee Ermey', 'Sarah Freeman', 'Penn Jillette']",1995,1145.180110
1,8844,N/D,65000000.0,"['Adventure', 'Fantasy', 'Family']",en,"When siblings Judy and Peter discover an enchanted board game that opens the door to a magical world, they unwittingly invite Alan -- an adult who's been trapped inside the game for 26 years -- into their living room. Alan's only hope for freedom is to finish the game, which proves risky as all three find themselves running from giant rhinoceroses, evil monkeys and other terrifying creatures.",17.015540,"['TriStar Pictures', 'Teitler Film', 'Interscope Communications']",['United States of America'],1995-12-15,262797249.0,104.0,"['English', 'Français']",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,Joe Johnston,"['Robin Williams', 'Jonathan Hyde', 'Kirsten Dunst', 'Bradley Pierce', 'Bonnie Hunt', 'Bebe Neuwirth', 'David Alan Grier', 'Patricia Clarkson', 'Adam Hann-Byrd', 'Laura Bell Bundy', 'James Handy', 'Gillian Barber', 'Brandon Obray', 'Cyrus Thiedeke', 'Gary Joseph Thorup', 'Leonard Zola', 'Lloyd Berry', 'Malcolm Stewart', 'Annabel Kershaw', 'Darryl Henriques', 'Robyn Driscoll', 'Peter Bryant', 'Sarah Gilson', 'Florica Vlad', 'June Lion', 'Brenda Lockmuller']",1995,304.303460
2,15602,Grumpy Old Men Collection,0.0,"['Romance', 'Comedy']",en,"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.",11.712900,"['Warner Bros.', 'Lancaster Gate']",['United States of America'],1995-12-22,0.0,101.0,['English'],Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,6.5,92.0,Howard Deutch,"['Walter Matthau', 'Jack Lemmon', 'Ann-Margret', 'Sophia Loren', 'Daryl Hannah', 'Burgess Meredith', 'Kevin Pollak']",1995,0.000000
3,31357,N/D,16000000.0,"['Comedy', 'Drama', 'Romance']",en,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusive ""good man"" to break a string of less-than-stellar lovers. Friends and confidants Vannah, Bernie, Glo and Robin talk it all out, determined to find a better way to breathe.",3.859495,['Twentieth Century Fox Film Corporation'],['United States of America'],1995-12-22,81452156.0,127.0,['English'],Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,6.1,34.0,Forest Whitaker,"['Whitney Houston', 'Angela Bassett', 'Loretta Devine', 'Lela Rochon', 'Gregory Hines', 'Dennis Haysbert', 'Michael Beach', 'Mykelti Williamson', 'Lamont Johnson', 'Wesley Snipes']",1995,409.075975
4,11862,Father of the Bride Collection,0.0,['Comedy'],en,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she's pregnant ... and that George's wife, Nina, is expecting too. He was planning on selling their home, but that's a pl

In [43]:
df.describe()

,budget,popularity,revenue,runtime,vote_average,vote_count,release_year,return
count,4.219600e+04,42196.000000,4.219600e+04,42196.000000,42196.000000,42196.000000,42196.000000,4.219600e+04
mean,4.160088e+06,2.914097,1.116670e+07,93.853302,5.631261,109.391720,1991.914399,7.095609e+04
std,1.721499e+07,5.744661,6.377178e+07,37.578123,1.921395,489.341742,23.839319,7.745654e+06
min,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1874.000000,-1.000000e+02
25%,0.000000e+00,0.387424,0.000000e+00,85.000000,5.000000,3.000000,1978.000000,0.000000e+00
50%,0.000000e+00,1.136089,0.000000e+00,95.000000,6.000000,10.000000,2001.000000,0.000000e+00
75%,0.000000e+00,3.717885,0.000000e+00,106.000000,6.800000,35.000000,2010.000000,0.000000e+00
max,3.800000e+08,547.488300,2.787965e+09,1256.000000,10.000000,14075.000000,2020.000000,1.239638e+09


In [44]:
df_reducido = df[['belongs_to_collection', 'genres',
       'overview',  'production_companies',
       'production_countries',
         'tagline', 'title',  'director', 'actors_names']].copy()

In [45]:
series_genres = pd.DataFrame(df_reducido['genres'])
series_genres.replace('N/D', '[]', inplace=True)
series_genres.head()

,genres
id,
862,"['Animation', 'Comedy', 'Family']"
8844,"['Adventure', 'Fantasy', 'Family']"
15602,"['Romance', 'Comedy']"
31357,"['Comedy', 'Drama', 'Romance']"
11862,['Comedy']


In [46]:
generos_unicos = []
for i in range(0, series_genres.shape[0]):
    fila = series_genres.iloc[i, 0]
    fila = eval(fila)
    for genero in fila:
        if genero not in generos_unicos:
            generos_unicos.append(genero)
print(generos_unicos)
    

['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History', 'Science Fiction', 'Mystery', 'War', 'Foreign', 'Music', 'Documentary', 'Western', 'TV Movie']


In [47]:
diccionario_generos = {}
for genero in generos_unicos:
    diccionario_generos[genero] = []


for i in range(0, series_genres.shape[0]):
    fila = series_genres.iloc[i, 0]
    fila = eval(fila)
    if len(fila) == 0:
        for key in diccionario_generos:
            diccionario_generos[key].append(0)
    else:
        for key in diccionario_generos:
            if key in fila:
                diccionario_generos[key].append(1)
            else:
                diccionario_generos[key].append(0)

generos_dummies = pd.DataFrame(diccionario_generos)
generos_dummies.head(3)

,Animation,Comedy,Family,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,History,Science Fiction,Mystery,War,Foreign,Music,Documentary,Western,TV Movie
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
generos_dummies['vote_average'] = df['vote_average'].values
generos_dummies['popularity'] = df['popularity'].values

In [49]:
generos_dummies.popularity.max

<bound method NDFrame._add_numeric_operations.<locals>.max of 0        21.946943
1        17.015540
2        11.712900
3         3.859495
4         8.387519
           ...    
42191     0.386450
42192     0.661558
42193     0.178241
42194     0.003503
42195     0.163015
Name: popularity, Length: 42196, dtype: float64>

In [58]:
df_index = list(df.index)
dicc_index = {'id': df_index}
df_index = pd.DataFrame(dicc_index)
generos_dummies['id'] = df_index
generos_dummies.to_csv('data/generos_dummies')